In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
# import pickle
import time
# from tqdm import tqdm

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
import sys
sys.path.append('/Users/zero/xgboost/python-package')
import xgboost as xgb
# import lightgbm as lgb

from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.metrics import log_loss

/opt/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# csv融合

# xgb

In [4]:
train = pd.read_csv('train_zero.csv')

train.shape
train.head()

(1597063, 341)

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,user_age_level,user_occupation_id,user_star_level,context_id,context_timestamp,context_page_id,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,time,day,hour,minute,user_query_day,user_query_day_hour,user_query_minute,item_category_split_count,item_property_split_count,pred_category_split_count,pred_property_split_count,item_pred_category_score,item_pred_property_score,item_pred_category_score_item%,item_pred_property_score_item%,item_pred_category_score_pred%,item_pred_property_score_pred%,user_shop_count,user_item_count,user_context_count,user_shop_trade_count,user_item_trade_count,user_context_trade_count,user_brand_count,user_city_count,user_brand_trade_count,user_city_trade_count,item_occupation_count,item_age_count,item_gender_count,item_occupation_trade_count,item_age_trade_count,item_gender_trade_count,shop_score,item_score,item_score2,item_score3,item_hot,user_hot,shop_hot,brand_hot,occupation_hot,city_hot,item_trade_hot,user_trade_hot,shop_trade_hot,brand_trade_hot,occupation_trade_hot,city_trade_hot,user_sell_power,user_sell_power_mean,user_sell_power_mean_item_minus,user_sell_trade_power,user_sell_trade_power_mean,user_sell_trade_power_mean_item_minus,yesterday,yesterhour,user_day_query,user_yesterday_query,user_hour_query,user_yesterhour_query,user_minute_query,item_day_query,item_hour_query,item_minute_query,user_item_day_query,user_item_hour_query,user_item_minute_query,min_time,user_is_his,item_is_his,brand_is_his,user_is_trade_his,item_is_trade_his,brand_is_trade_his,user_count,user_day_count,user_hour_count,user_minute_count,item_trade_percent,brand_trade_percent,user_trade_percent,occupation_trade_percent,is_home,shop_item_count,brand_item_count,city_brand_count,city_item_count,shop_item_count_his,brand_item_count_his,city_brand_count_his,city_item_count_his,user_next_count,user_last_count,user_next_time,user_last_time,next_time_sub,last_time_sub,user_item_num,user_shop_num,user_item_day_num,user_shop_day_num,user_item_num_next,user_item_num_last,user_shop_num_next,user_shop_num_last,user_item_next_time,user_item_last_time,user_shop_next_time,user_shop_last_time,user_item_next_time_sub,user_item_last_time_sub,user_shop_next_time_sub,user_shop_last_time_sub,user_fisrt_num,user_item_fisrt_num,user_shop_fisrt_num,user_count_first_time,user_item_count_first_time,user_shop_count_first_time,user_count_first_time_sub,user_item_count_first_time_sub,user_shop_count_first_time_sub,max_click,is_last_click,max_user_item_click,is_last_user_item_click,max_click_time,max_user_item_click_time,max_click_time_sub,max_user_item_click_time_sub,instance_id_6,time_6,day_6,hour_6,minute_6,user_query_day_6,user_query_day_hour_6,user_query_minute_6,item_category_split_count_6,item_property_split_count_6,pred_category_split_count_6,pred_property_split_count_6,item_pred_category_score_6,item_pred_property_score_6,item_pred_category_score_item%_6,item_pred_property_score_item%_6,item_pred_category_score_pred%_6,item_pred_property_score_pred%_6,user_shop_count_6,user_item_count_6,user_context_count_6,user_shop_trade_count_6,user_item_trade_count_6,user_context_trade_count_6,user_brand_count_6,user_city_count_6,user_brand_trade_count_6,user_city_trade_count_6,item_occupation_count_6,item_age_count_6,item_gender_count_6,item_occupation_trade_count_6,item_age_trade_count_6,item_gender_trade_count_6,shop_score_6,item_score_6,item_score2_6,item_score3_6,item_hot_6,user_hot_6,shop_hot_6,brand_hot_6,occupation_hot_6,city_hot_6,item_trade_hot_6,user_trade_hot_6,shop_trade_hot_6,brand_trade_hot_6,occupation_trade_hot_6,city_trade_hot_6,user_sell_power_6,user_sell_power_mean_6,user_sell_power_mean_item_minus_6,user_sell_trade_power_6,user_sell_trade_power_mean_6,user_sell_trade_power_mean_item_minus_6,yesterday_6,yes

In [5]:
test = train[train.is_trade == -1]
train = train[train.is_trade != -1]

test.shape
train.shape

(519888, 341)

(1077175, 341)

In [6]:
origin = [
    'item_price_level', 'item_sales_level', 'item_collected_level',
    'item_pv_level', 'user_gender_id', 'user_age_level', 'user_occupation_id',
    'user_star_level', 'context_page_id', 'shop_review_num_level',
    'shop_review_positive_rate', 'shop_star_level', 'shop_score_service',
    'shop_score_delivery', 'shop_score_description'
]

features_pre5 = [
    'day', 'hour', 'minute', 'user_query_day', 'user_query_day_hour',
    'user_query_minute', 'item_category_split_count',
    'item_property_split_count', 'pred_category_split_count',
    'pred_property_split_count', 'item_pred_category_score',
    'item_pred_property_score', 'item_pred_category_score_item%',
    'item_pred_property_score_item%', 'item_pred_category_score_pred%',
    'item_pred_property_score_pred%', 'user_shop_count', 'user_item_count',
    'user_context_count', 'user_shop_trade_count', 'user_item_trade_count',
    'user_context_trade_count', 'user_brand_count', 'user_city_count',
    'user_brand_trade_count', 'user_city_trade_count', 'item_occupation_count',
    'item_age_count', 'item_gender_count', 'item_occupation_trade_count',
    'item_age_trade_count', 'item_gender_trade_count', 'shop_score',
    'item_score', 'item_score2', 'item_score3', 'item_hot', 'user_hot',
    'shop_hot', 'brand_hot', 'occupation_hot', 'city_hot', 'item_trade_hot',
    'user_trade_hot', 'shop_trade_hot', 'brand_trade_hot',
    'occupation_trade_hot', 'city_trade_hot', 'user_sell_power',
    'user_sell_power_mean', 'user_sell_power_mean_item_minus',
    'user_sell_trade_power', 'user_sell_trade_power_mean',
    'user_sell_trade_power_mean_item_minus', 'yesterday', 'yesterhour',
    'user_day_query', 'user_yesterday_query', 'user_hour_query',
    'user_yesterhour_query', 'user_minute_query', 'item_day_query',
    'item_hour_query', 'item_minute_query', 'user_item_day_query',
    'user_item_hour_query', 'user_item_minute_query', 'min_time',
    'user_is_his', 'item_is_his', 'brand_is_his', 'user_is_trade_his',
    'item_is_trade_his', 'brand_is_trade_his', 'user_count', 'user_day_count',
    'user_hour_count', 'user_minute_count', 'item_trade_percent',
    'brand_trade_percent', 'user_trade_percent', 'occupation_trade_percent',
    'is_home', 'shop_item_count', 'brand_item_count', 'city_brand_count',
    'city_item_count', 'shop_item_count_his', 'brand_item_count_his',
    'city_brand_count_his', 'city_item_count_his', 'next_time_sub',
    'last_time_sub', 'user_item_num', 'user_shop_num', 'user_item_day_num',
    'user_shop_day_num', 'user_item_next_time_sub', 'user_item_last_time_sub',
    'user_shop_next_time_sub', 'user_shop_last_time_sub',
    'user_count_first_time_sub', 'user_item_count_first_time_sub',
    'user_shop_count_first_time_sub', 'max_click', 'is_last_click',
    'max_user_item_click', 'is_last_user_item_click', 'max_click_time_sub',
    'max_user_item_click_time_sub'
]

features_6 = [
    'user_shop_count_6', 'user_item_count_6', 'user_context_count_6',
    'user_shop_trade_count_6', 'user_item_trade_count_6',
    'user_context_trade_count_6', 'user_brand_count_6', 'user_city_count_6',
    'user_brand_trade_count_6', 'user_city_trade_count_6',
    'item_occupation_count_6', 'item_age_count_6', 'item_gender_count_6',
    'item_occupation_trade_count_6', 'item_age_trade_count_6',
    'item_gender_trade_count_6', 'shop_score_6', 'item_score_6',
    'item_score2_6', 'item_score3_6', 'item_hot_6', 'user_hot_6', 'shop_hot_6',
    'brand_hot_6', 'occupation_hot_6', 'city_hot_6', 'item_trade_hot_6',
    'user_trade_hot_6', 'shop_trade_hot_6', 'brand_trade_hot_6',
    'occupation_trade_hot_6', 'city_trade_hot_6', 'user_sell_power_6',
    'user_sell_power_mean_6', 'user_sell_power_mean_item_minus_6',
    'user_sell_trade_power_6', 'user_sell_trade_power_mean_6',
    'user_sell_trade_power_mean_item_minus_6', 'yesterday_6', 'yesterhour_6',
    'user_day_query_6', 'user_yesterday_query_6', 'user_hour_query_6',
    'user_yesterhour_query_6', 'user_minute_query_6', 'item_day_query_6',
    'item_hour_query_6', 'item_minute_query_6', 'user_item_day_query_6',
    'user_item_hour_query_6', 'user_item_minute_query_6', 'min_time_6',
    'user_is_his_6', 'item_is_his_6', 'brand_is_his_6', 'user_is_trade_his_6',
    'item_is_trade_his_6', 'brand_is_trade_his_6', 'user_count_6',
    'user_day_count_6', 'user_hour_count_6', 'user_minute_count_6',
    'item_trade_percent_6', 'brand_trade_percent_6', 'user_trade_percent_6',
    'occupation_trade_percent_6', 'is_home_6', 'shop_item_count_6',
    'brand_item_count_6', 'city_brand_count_6', 'city_item_count_6',
    'shop_item_count_his_6', 'brand_item_count_his_6',
    'city_brand_count_his_6', 'city_item_count_his_6', 'next_time_sub_6',
    'last_time_sub_6', 'user_item_num_6', 'user_shop_num_6',
    'user_item_day_num_6', 'user_shop_day_num_6', 'user_item_next_time_sub_6',
    'user_item_last_time_sub_6', 'user_shop_next_time_sub_6',
    'user_shop_last_time_sub_6', 'user_count_first_time_sub_6',
    'user_item_count_first_time_sub_6', 'user_shop_count_first_time_sub_6',
    'max_click_6', 'is_last_click_6', 'max_user_item_click_6',
    'is_last_user_item_click_6', 'max_click_time_sub_6',
    'max_user_item_click_time_sub_6'
]

features_fulltime = [
    'user_count_full_time', 'user_day_count_full_time',
    'user_hour_count_full_time', 'user_minute_count_full_time',
    'next_time_sub_full_time', 'last_time_sub_full_time',
    'user_item_num_full_time', 'user_shop_num_full_time',
    'user_item_day_num_full_time', 'user_shop_day_num_full_time',
    'user_item_next_time_sub_full_time', 'user_item_last_time_sub_full_time',
    'user_shop_next_time_sub_full_time', 'user_shop_last_time_sub_full_time',
    'user_count_first_time_sub_full_time',
    'user_item_count_first_time_sub_full_time',
    'user_shop_count_first_time_sub_full_time', 'max_click_full_time',
    'is_last_click_full_time', 'max_user_item_click_full_time',
    'is_last_user_item_click_full_time', 'max_user_item_click_time_sub_full_time'
]

target = ['is_trade']

In [7]:
good_features = [
    'brand_hot', 'brand_is_trade_his', 'brand_item_count',
    'brand_item_count_his', 'brand_trade_hot', 'brand_trade_percent',
    'city_brand_count', 'city_brand_count_his', 'city_hot', 'city_item_count',
    'city_item_count_his', 'city_trade_hot', 'hour', 'item_age_trade_count',
    'item_category_split_count', 'item_day_query', 'item_gender_count',
    'item_gender_trade_count', 'item_hot', 'item_hour_query',
    'item_is_trade_his', 'item_minute_query', 'item_occupation_count',
    'item_occupation_trade_count', 'item_pred_category_score',
    'item_pred_category_score_item%', 'item_pred_property_score',
    'item_pred_property_score_item%', 'item_pred_property_score_pred%',
    'item_property_split_count', 'item_score2', 'item_trade_hot',
    'item_trade_percent', 'last_time_sub', 'min_time', 'next_time_sub',
    'shop_hot', 'shop_item_count', 'shop_item_count_his', 'shop_score',
    'shop_trade_hot', 'user_brand_count', 'user_city_count', 'user_count',
    'user_day_count', 'user_day_query', 'user_hot', 'user_hour_count',
    'user_hour_query', 'user_is_his', 'user_item_count', 'user_item_day_num',
    'user_item_day_query', 'user_item_hour_query', 'user_item_next_time_sub',
    'user_item_num', 'user_minute_count', 'user_minute_query',
    'user_query_day', 'user_query_day_hour', 'user_query_minute',
    'user_sell_power', 'user_sell_power_mean',
    'user_sell_power_mean_item_minus', 'user_shop_count', 'user_shop_day_num',
    'user_shop_num', 'yesterhour', 'user_item_last_time_sub',
    'user_shop_next_time_sub', 'user_shop_last_time_sub',
    'user_count_first_time_sub', 'user_item_count_first_time_sub',
    'user_shop_count_first_time_sub', 'max_click', 'is_last_click',
    'max_user_item_click', 'is_last_user_item_click', 'max_click_time_sub',
    'max_user_item_click_time_sub', 'user_shop_count_6', 'user_item_count_6',
    'user_brand_count_6', 'user_city_count_6', 'item_occupation_count_6',
    'item_gender_count_6', 'item_occupation_trade_count_6',
    'item_age_trade_count_6', 'item_gender_trade_count_6', 'shop_score_6',
    'item_score2_6', 'item_hot_6', 'user_hot_6', 'shop_hot_6', 'brand_hot_6',
    'city_hot_6', 'item_trade_hot_6', 'shop_trade_hot_6', 'brand_trade_hot_6',
    'city_trade_hot_6', 'user_sell_power_6', 'user_sell_power_mean_6',
    'user_sell_power_mean_item_minus_6', 'yesterhour_6', 'user_day_query_6',
    'user_hour_query_6', 'user_minute_query_6', 'item_day_query_6',
    'item_hour_query_6', 'item_minute_query_6', 'user_item_day_query_6',
    'user_item_hour_query_6', 'min_time_6', 'user_is_his_6',
    'item_is_trade_his_6', 'brand_is_trade_his_6', 'user_count_6',
    'user_day_count_6', 'user_hour_count_6', 'user_minute_count_6',
    'item_trade_percent_6', 'brand_trade_percent_6', 'shop_item_count_6',
    'brand_item_count_6', 'city_brand_count_6', 'city_item_count_6',
    'shop_item_count_his_6', 'brand_item_count_his_6',
    'city_brand_count_his_6', 'city_item_count_his_6', 'next_time_sub_6',
    'last_time_sub_6', 'user_item_num_6', 'user_shop_num_6',
    'user_item_day_num_6', 'user_shop_day_num_6', 'user_item_next_time_sub_6',
    'user_item_last_time_sub_6', 'user_shop_next_time_sub_6',
    'user_shop_last_time_sub_6', 'user_count_first_time_sub_6',
    'user_item_count_first_time_sub_6', 'user_shop_count_first_time_sub_6',
    'max_click_6', 'is_last_click_6', 'max_user_item_click_6',
    'is_last_user_item_click_6', 'max_click_time_sub_6',
    'max_user_item_click_time_sub_6', 'user_count_full_time',
    'user_day_count_full_time', 'user_hour_count_full_time',
    'user_minute_count_full_time', 'next_time_sub_full_time',
    'last_time_sub_full_time', 'user_item_num_full_time',
    'user_shop_num_full_time', 'user_item_day_num_full_time',
    'user_shop_day_num_full_time', 'user_item_next_time_sub_full_time',
    'user_item_last_time_sub_full_time', 'user_shop_next_time_sub_full_time',
    'user_shop_last_time_sub_full_time', 'user_count_first_time_sub_full_time',
    'user_item_count_first_time_sub_full_time',
    'user_shop_count_first_time_sub_full_time', 'max_click_full_time',
    'is_last_click_full_time', 'max_user_item_click_full_time',
    'is_last_user_item_click_full_time',
    'max_user_item_click_time_sub_full_time', 'user_shop_count_pre5_6_sub',
    'user_item_count_pre5_6_sub', 'user_brand_count_pre5_6_sub',
    'user_brand_trade_count_pre5_6_sub', 'item_occupation_count_pre5_6_sub',
    'item_gender_count_pre5_6_sub', 'item_occupation_trade_count_pre5_6_sub',
    'item_age_trade_count_pre5_6_sub', 'item_gender_trade_count_pre5_6_sub',
    'item_hot_pre5_6_sub', 'user_hot_pre5_6_sub', 'shop_hot_pre5_6_sub',
    'brand_hot_pre5_6_sub', 'city_hot_pre5_6_sub', 'item_trade_hot_pre5_6_sub',
    'user_trade_hot_pre5_6_sub', 'shop_trade_hot_pre5_6_sub',
    'brand_trade_hot_pre5_6_sub', 'city_trade_hot_pre5_6_sub',
    'user_sell_power_pre5_6_sub', 'user_sell_power_mean_pre5_6_sub',
    'user_sell_power_mean_item_minus_pre5_6_sub',
    'brand_is_trade_his_pre5_6_sub', 'item_trade_percent_pre5_6_sub',
    'brand_trade_percent_pre5_6_sub', 'shop_item_count_his_pre5_6_sub',
    'brand_item_count_his_pre5_6_sub', 'city_brand_count_his_pre5_6_sub',
    'city_item_count_his_pre5_6_sub', 'next_time_sub_pre5_6_sub',
    'user_item_next_time_sub_pre5_6_sub', 'user_shop_next_time_sub_pre5_6_sub'
]

len(good_features)

203

In [8]:
sub_features = []
for i in features_6:
    tmp = str(i[:-2] + '_pre5_6_sub')
    train[tmp] = abs(train[i[:-2]] - train[i])
    sub_features.append(tmp)
len(sub_features)

94

In [ ]:
predictors = features_pre5 + features_6 + features_fulltime + sub_features
len(origin)
len(predictors)

In [14]:
#baseline
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

tmp_predictors = origin.copy()
len(tmp_predictors)
xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])

cv = xgb.cv(
    params,
    dtrain=xgbtrain,
    num_boost_round=800,
    nfold=3,
    early_stopping_rounds=50,
    seed=201803)
    
print('Baseline->' + str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))
    
# Baseline:0.177976666667(400粗筛)

15

Baseline->800:0.177572


In [ ]:
#cv
params = {
    'objective': 'binary:logistic',
    'eta': 0.05,
    'colsample_bytree': 0.886,
    'min_child_weight': 1.6,
    'max_depth': 4,
    'subsample': 0.886,
    'gamma': 0.1,
    'lambda': 10,
    'verbose_eval': True,
    'eval_metric': 'logloss',
    'seed': 201803,
    'missing': -1
}

# feature_peoformance = {}

for i in enumerate(good_features):
    if i[1] not in feature_peoformance.keys():
        tmp_predictors = origin.copy()
        tmp_predictors.append(i[1])
        xgbtrain = xgb.DMatrix(train[tmp_predictors], train[target])

        cv = xgb.cv(
            params,
            dtrain=xgbtrain,
            num_boost_round=800,
            nfold=3,
            early_stopping_rounds=50,
            seed=201803)
    
        feature_peoformance[i[1]] = cv.iloc[-1, :]['test-logloss-mean']
        print(str(i[0]) + ':' + str(i[1]) + '->' + str(cv.shape[0]) + ':' + str(cv.iloc[-1, :]['test-logloss-mean']))
    
# Baseline:0.177976666667(400粗筛)

136:user_item_next_time_sub_6->800:0.176198333333
137:user_item_last_time_sub_6->800:0.177000666667
138:user_shop_next_time_sub_6->800:0.176474333333
139:user_shop_last_time_sub_6->800:0.177078666667
140:user_count_first_time_sub_6->800:0.177509333333
141:user_item_count_first_time_sub_6->800:0.177009666667
142:user_shop_count_first_time_sub_6->800:0.177082666667
143:max_click_6->800:0.176297666667
144:is_last_click_6->800:0.175338
145:max_user_item_click_6->800:0.177459666667
146:is_last_user_item_click_6->800:0.176073666667
147:max_click_time_sub_6->800:0.175065666667
148:max_user_item_click_time_sub_6->800:0.176054666667
149:user_count_full_time->800:0.177534
150:user_day_count_full_time->800:0.177534
151:user_hour_count_full_time->800:0.177546666667
152:user_minute_count_full_time->800:0.177484333333
153:next_time_sub_full_time->800:0.174306333333
154:last_time_sub_full_time->800:0.17738
155:user_item_num_full_time->800:0.177020666667
156:user_shop_num_full_time->800:0.177099333333

In [13]:
len(feature_peoformance)
feature_peoformance

203

{'brand_hot': 0.17749033333333333,
 'brand_hot_6': 0.17750200000000002,
 'brand_hot_pre5_6_sub': 0.17749133333333333,
 'brand_is_trade_his': 0.17756400000000003,
 'brand_is_trade_his_6': 0.17755566666666667,
 'brand_is_trade_his_pre5_6_sub': 0.17756833333333333,
 'brand_item_count': 0.17752866666666667,
 'brand_item_count_6': 0.17752866666666667,
 'brand_item_count_his': 0.17751066666666668,
 'brand_item_count_his_6': 0.17752366666666666,
 'brand_item_count_his_pre5_6_sub': 0.17752533333333331,
 'brand_trade_hot': 0.17747533333333335,
 'brand_trade_hot_6': 0.17751866666666663,
 'brand_trade_hot_pre5_6_sub': 0.17748833333333333,
 'brand_trade_percent': 0.1773153333333333,
 'brand_trade_percent_6': 0.17744266666666667,
 'brand_trade_percent_pre5_6_sub': 0.1773773333333333,
 'city_brand_count': 0.17745633333333333,
 'city_brand_count_6': 0.17745633333333333,
 'city_brand_count_his': 0.177455,
 'city_brand_count_his_6': 0.17745333333333332,
 'city_brand_count_his_pre5_6_sub': 0.17744533333

In [15]:
baseline = 0.177572
good_features = []

for i in feature_peoformance.keys():
    if feature_peoformance[i] < baseline:
        good_features.append(i)
len(good_features)

197

In [16]:
good_features

['brand_hot',
 'brand_is_trade_his',
 'brand_item_count',
 'brand_item_count_his',
 'brand_trade_hot',
 'brand_trade_percent',
 'city_brand_count',
 'city_brand_count_his',
 'city_hot',
 'city_item_count',
 'city_item_count_his',
 'city_trade_hot',
 'hour',
 'item_age_trade_count',
 'item_category_split_count',
 'item_day_query',
 'item_gender_count',
 'item_gender_trade_count',
 'item_hot',
 'item_is_trade_his',
 'item_minute_query',
 'item_occupation_count',
 'item_occupation_trade_count',
 'item_pred_category_score',
 'item_pred_category_score_item%',
 'item_pred_property_score',
 'item_pred_property_score_item%',
 'item_pred_property_score_pred%',
 'item_property_split_count',
 'item_trade_hot',
 'item_trade_percent',
 'last_time_sub',
 'min_time',
 'next_time_sub',
 'shop_hot',
 'shop_item_count',
 'shop_item_count_his',
 'shop_score',
 'shop_trade_hot',
 'user_brand_count',
 'user_city_count',
 'user_count',
 'user_day_count',
 'user_day_query',
 'user_hot',
 'user_hour_count',
 

In [ ]:
# 细筛
{'brand_hot': 0.17749033333333333,
 'brand_hot_6': 0.17750200000000002,
 'brand_hot_pre5_6_sub': 0.17749133333333333,
 'brand_is_trade_his': 0.17756400000000003,
 'brand_is_trade_his_6': 0.17755566666666667,
 'brand_is_trade_his_pre5_6_sub': 0.17756833333333333,
 'brand_item_count': 0.17752866666666667,
 'brand_item_count_6': 0.17752866666666667,
 'brand_item_count_his': 0.17751066666666668,
 'brand_item_count_his_6': 0.17752366666666666,
 'brand_item_count_his_pre5_6_sub': 0.17752533333333331,
 'brand_trade_hot': 0.17747533333333335,
 'brand_trade_hot_6': 0.17751866666666663,
 'brand_trade_hot_pre5_6_sub': 0.17748833333333333,
 'brand_trade_percent': 0.1773153333333333,
 'brand_trade_percent_6': 0.17744266666666667,
 'brand_trade_percent_pre5_6_sub': 0.1773773333333333,
 'city_brand_count': 0.17745633333333333,
 'city_brand_count_6': 0.17745633333333333,
 'city_brand_count_his': 0.177455,
 'city_brand_count_his_6': 0.17745333333333332,
 'city_brand_count_his_pre5_6_sub': 0.17744533333333334,
 'city_hot': 0.17745533333333333,
 'city_hot_6': 0.17746799999999999,
 'city_hot_pre5_6_sub': 0.17745466666666668,
 'city_item_count': 0.177458,
 'city_item_count_6': 0.177458,
 'city_item_count_his': 0.17744866666666667,
 'city_item_count_his_6': 0.17747366666666667,
 'city_item_count_his_pre5_6_sub': 0.17743899999999999,
 'city_trade_hot': 0.17744166666666669,
 'city_trade_hot_6': 0.17745200000000003,
 'city_trade_hot_pre5_6_sub': 0.17746100000000001,
 'hour': 0.17754266666666665,
 'is_last_click': 0.17533799999999999,
 'is_last_click_6': 0.17533799999999999,
 'is_last_click_full_time': 0.17533799999999999,
 'is_last_user_item_click': 0.17607366666666668,
 'is_last_user_item_click_6': 0.17607366666666668,
 'is_last_user_item_click_full_time': 0.17607366666666668,
 'item_age_trade_count': 0.17739466666666667,
 'item_age_trade_count_6': 0.17754866666666666,
 'item_age_trade_count_pre5_6_sub': 0.17738933333333332,
 'item_category_split_count': 0.17753066666666664,
 'item_day_query': 0.17744866666666667,
 'item_day_query_6': 0.17744866666666667,
 'item_gender_count': 0.17754333333333336,
 'item_gender_count_6': 0.177563,
 'item_gender_count_pre5_6_sub': 0.17755100000000001,
 'item_gender_trade_count': 0.17733033333333334,
 'item_gender_trade_count_6': 0.17750333333333335,
 'item_gender_trade_count_pre5_6_sub': 0.17736666666666667,
 'item_hot': 0.17746766666666666,
 'item_hot_6': 0.17746666666666666,
 'item_hot_pre5_6_sub': 0.17746533333333334,
 'item_hour_query': 0.17757833333333331,
 'item_hour_query_6': 0.17757833333333331,
 'item_is_trade_his': 0.17752433333333331,
 'item_is_trade_his_6': 0.17750299999999999,
 'item_minute_query': 0.17754633333333336,
 'item_minute_query_6': 0.17754633333333336,
 'item_occupation_count': 0.17755799999999999,
 'item_occupation_count_6': 0.17758333333333332,
 'item_occupation_count_pre5_6_sub': 0.17754733333333336,
 'item_occupation_trade_count': 0.17740866666666666,
 'item_occupation_trade_count_6': 0.17754133333333333,
 'item_occupation_trade_count_pre5_6_sub': 0.17742266666666665,
 'item_pred_category_score': 0.17754066666666665,
 'item_pred_category_score_item%': 0.17752066666666666,
 'item_pred_property_score': 0.17755766666666664,
 'item_pred_property_score_item%': 0.17755866666666664,
 'item_pred_property_score_pred%': 0.177562,
 'item_property_split_count': 0.17748533333333336,
 'item_score2': 0.17757233333333333,
 'item_score2_6': 0.17757233333333333,
 'item_trade_hot': 0.17724233333333336,
 'item_trade_hot_6': 0.17744699999999999,
 'item_trade_hot_pre5_6_sub': 0.17730233333333334,
 'item_trade_percent': 0.17700000000000002,
 'item_trade_percent_6': 0.17738800000000002,
 'item_trade_percent_pre5_6_sub': 0.17728166666666667,
 'last_time_sub': 0.17738000000000001,
 'last_time_sub_6': 0.17738000000000001,
 'last_time_sub_full_time': 0.17738000000000001,
 'max_click': 0.17629766666666669,
 'max_click_6': 0.17629766666666669,
 'max_click_full_time': 0.17629766666666669,
 'max_click_time_sub': 0.17506566666666667,
 'max_click_time_sub_6': 0.17506566666666667,
 'max_user_item_click': 0.17745966666666668,
 'max_user_item_click_6': 0.17745966666666668,
 'max_user_item_click_full_time': 0.17745966666666668,
 'max_user_item_click_time_sub': 0.17605466666666666,
 'max_user_item_click_time_sub_6': 0.17605466666666666,
 'max_user_item_click_time_sub_full_time': 0.17605466666666666,
 'min_time': 0.17753033333333335,
 'min_time_6': 0.17753033333333335,
 'next_time_sub': 0.17421400000000001,
 'next_time_sub_6': 0.17425566666666667,
 'next_time_sub_full_time': 0.17430633333333334,
 'next_time_sub_pre5_6_sub': 0.17703599999999997,
 'shop_hot': 0.177457,
 'shop_hot_6': 0.17746700000000001,
 'shop_hot_pre5_6_sub': 0.17748033333333332,
 'shop_item_count': 0.17748766666666668,
 'shop_item_count_6': 0.17748766666666668,
 'shop_item_count_his': 0.17748700000000003,
 'shop_item_count_his_6': 0.17749733333333331,
 'shop_item_count_his_pre5_6_sub': 0.17752233333333334,
 'shop_score': 0.17755666666666667,
 'shop_score_6': 0.17755666666666667,
 'shop_trade_hot': 0.17745200000000003,
 'shop_trade_hot_6': 0.17750633333333332,
 'shop_trade_hot_pre5_6_sub': 0.177455,
 'user_brand_count': 0.17755666666666667,
 'user_brand_count_6': 0.17749666666666664,
 'user_brand_count_pre5_6_sub': 0.17749733333333331,
 'user_brand_trade_count_pre5_6_sub': 0.17758166666666667,
 'user_city_count': 0.17755033333333334,
 'user_city_count_6': 0.17754933333333334,
 'user_count': 0.177534,
 'user_count_6': 0.177534,
 'user_count_first_time_sub': 0.17750933333333332,
 'user_count_first_time_sub_6': 0.17750933333333332,
 'user_count_first_time_sub_full_time': 0.17750933333333332,
 'user_count_full_time': 0.177534,
 'user_day_count': 0.177534,
 'user_day_count_6': 0.177534,
 'user_day_count_full_time': 0.177534,
 'user_day_query': 0.17629766666666669,
 'user_day_query_6': 0.17629766666666669,
 'user_hot': 0.17726033333333333,
 'user_hot_6': 0.17753433333333332,
 'user_hot_pre5_6_sub': 0.17735133333333333,
 'user_hour_count': 0.17754666666666666,
 'user_hour_count_6': 0.17754666666666666,
 'user_hour_count_full_time': 0.17754666666666666,
 'user_hour_query': 0.17644666666666664,
 'user_hour_query_6': 0.17644666666666664,
 'user_is_his': 0.17738366666666669,
 'user_is_his_6': 0.17755866666666664,
 'user_item_count': 0.17752400000000002,
 'user_item_count_6': 0.17747966666666667,
 'user_item_count_first_time_sub': 0.17700966666666665,
 'user_item_count_first_time_sub_6': 0.17700966666666665,
 'user_item_count_first_time_sub_full_time': 0.17700966666666665,
 'user_item_count_pre5_6_sub': 0.17741933333333329,
 'user_item_day_num': 0.17702066666666663,
 'user_item_day_num_6': 0.17702066666666663,
 'user_item_day_num_full_time': 0.17702066666666663,
 'user_item_day_query': 0.17745966666666668,
 'user_item_day_query_6': 0.17745966666666668,
 'user_item_hour_query': 0.17751966666666666,
 'user_item_hour_query_6': 0.17751966666666666,
 'user_item_last_time_sub': 0.17700066666666667,
 'user_item_last_time_sub_6': 0.17700066666666667,
 'user_item_last_time_sub_full_time': 0.17700066666666667,
 'user_item_next_time_sub': 0.17629066666666668,
 'user_item_next_time_sub_6': 0.17619833333333332,
 'user_item_next_time_sub_full_time': 0.17625033333333331,
 'user_item_next_time_sub_pre5_6_sub': 0.17734033333333332,
 'user_item_num': 0.17702066666666663,
 'user_item_num_6': 0.17702066666666663,
 'user_item_num_full_time': 0.17702066666666663,
 'user_minute_count': 0.17748433333333333,
 'user_minute_count_6': 0.17748433333333333,
 'user_minute_count_full_time': 0.17748433333333333,
 'user_minute_query': 0.17613666666666669,
 'user_minute_query_6': 0.17613666666666669,
 'user_query_day': 0.17629766666666669,
 'user_query_day_hour': 0.17644666666666664,
 'user_query_minute': 0.17613666666666669,
 'user_sell_power': 0.17727866666666667,
 'user_sell_power_6': 0.17754300000000001,
 'user_sell_power_mean': 0.17738133333333331,
 'user_sell_power_mean_6': 0.17757133333333333,
 'user_sell_power_mean_item_minus': 0.17738799999999999,
 'user_sell_power_mean_item_minus_6': 0.17755899999999999,
 'user_sell_power_mean_item_minus_pre5_6_sub': 0.17744000000000001,
 'user_sell_power_mean_pre5_6_sub': 0.17745833333333336,
 'user_sell_power_pre5_6_sub': 0.17737366666666665,
 'user_shop_count': 0.17753733333333332,
 'user_shop_count_6': 0.17746833333333334,
 'user_shop_count_first_time_sub': 0.17708266666666664,
 'user_shop_count_first_time_sub_6': 0.17708266666666664,
 'user_shop_count_first_time_sub_full_time': 0.17708266666666664,
 'user_shop_count_pre5_6_sub': 0.17744633333333334,
 'user_shop_day_num': 0.17709933333333336,
 'user_shop_day_num_6': 0.17709933333333336,
 'user_shop_day_num_full_time': 0.17709933333333336,
 'user_shop_last_time_sub': 0.17707866666666669,
 'user_shop_last_time_sub_6': 0.17707866666666669,
 'user_shop_last_time_sub_full_time': 0.17707866666666669,
 'user_shop_next_time_sub': 0.17655166666666666,
 'user_shop_next_time_sub_6': 0.17647433333333332,
 'user_shop_next_time_sub_full_time': 0.1765106666666667,
 'user_shop_next_time_sub_pre5_6_sub': 0.17735066666666666,
 'user_shop_num': 0.17709933333333336,
 'user_shop_num_6': 0.17709933333333336,
 'user_shop_num_full_time': 0.17709933333333336,
 'user_trade_hot_pre5_6_sub': 0.17755966666666664,
 'yesterhour': 0.17755666666666667,
 'yesterhour_6': 0.17755666666666667}

In [13]:
# 粗筛
{'brand_hot': 0.17791066666666666,
 'brand_hot_6': 0.17791733333333334,
 'brand_hot_pre5_6_sub': 0.17791566666666667,
 'brand_is_his': 0.17798933333333333,
 'brand_is_his_6': 0.17798933333333333,
 'brand_is_his_pre5_6_sub': 0.17798933333333333,
 'brand_is_trade_his': 0.17796900000000002,
 'brand_is_trade_his_6': 0.17797666666666667,
 'brand_is_trade_his_pre5_6_sub': 0.17796699999999999,
 'brand_item_count': 0.17793066666666668,
 'brand_item_count_6': 0.17793066666666668,
 'brand_item_count_his': 0.17792733333333333,
 'brand_item_count_his_6': 0.17792733333333333,
 'brand_item_count_his_pre5_6_sub': 0.17792299999999997,
 'brand_item_count_pre5_6_sub': 0.17798933333333333,
 'brand_trade_hot': 0.177875,
 'brand_trade_hot_6': 0.17790866666666663,
 'brand_trade_hot_pre5_6_sub': 0.17788733333333331,
 'brand_trade_percent': 0.17765699999999998,
 'brand_trade_percent_6': 0.17781933333333333,
 'brand_trade_percent_pre5_6_sub': 0.17775766666666667,
 'city_brand_count': 0.177875,
 'city_brand_count_6': 0.17787500000000001,
 'city_brand_count_his': 0.17787166666666665,
 'city_brand_count_his_6': 0.177871,
 'city_brand_count_his_pre5_6_sub': 0.17783000000000002,
 'city_brand_count_pre5_6_sub': 0.17798933333333333,
 'city_hot': 0.177891,
 'city_hot_6': 0.17788499999999999,
 'city_hot_pre5_6_sub': 0.17788266666666666,
 'city_item_count': 0.17787433333333333,
 'city_item_count_6': 0.17787433333333333,
 'city_item_count_his': 0.17786866666666667,
 'city_item_count_his_6': 0.17787266666666665,
 'city_item_count_his_pre5_6_sub': 0.17785733333333331,
 'city_item_count_pre5_6_sub': 0.17798933333333333,
 'city_trade_hot': 0.17784633333333333,
 'city_trade_hot_6': 0.17786533333333332,
 'city_trade_hot_pre5_6_sub': 0.17785833333333334,
 'day': 0.17798933333333333,
 'hour': 0.17796966666666666,
 'is_home': 0.17798733333333336,
 'is_home_6': 0.17798733333333336,
 'is_home_pre5_6_sub': 0.17798933333333333,
 'is_last_click': 0.17576866666666668,
 'is_last_click_6': 0.17576866666666668,
 'is_last_click_full_time': 0.17576866666666668,
 'is_last_click_pre5_6_sub': 0.17798933333333333,
 'is_last_user_item_click': 0.17653200000000002,
 'is_last_user_item_click_6': 0.17653200000000002,
 'is_last_user_item_click_full_time': 0.17653200000000002,
 'is_last_user_item_click_pre5_6_sub': 0.17798933333333333,
 'item_age_count': 0.17799566666666666,
 'item_age_count_6': 0.17798899999999998,
 'item_age_count_pre5_6_sub': 0.17799433333333334,
 'item_age_trade_count': 0.17774399999999999,
 'item_age_trade_count_6': 0.17793200000000001,
 'item_age_trade_count_pre5_6_sub': 0.17776400000000001,
 'item_category_split_count': 0.17794166666666666,
 'item_day_query': 0.17785933333333334,
 'item_day_query_6': 0.17785933333333334,
 'item_day_query_pre5_6_sub': 0.17798933333333333,
 'item_gender_count': 0.177947,
 'item_gender_count_6': 0.177956,
 'item_gender_count_pre5_6_sub': 0.17795633333333336,
 'item_gender_trade_count': 0.17771,
 'item_gender_trade_count_6': 0.17789599999999997,
 'item_gender_trade_count_pre5_6_sub': 0.17774333333333336,
 'item_hot': 0.17788166666666663,
 'item_hot_6': 0.17788300000000001,
 'item_hot_pre5_6_sub': 0.17790466666666668,
 'item_hour_query': 0.17796333333333333,
 'item_hour_query_6': 0.17796333333333333,
 'item_hour_query_pre5_6_sub': 0.17798933333333333,
 'item_is_his': 0.17798166666666668,
 'item_is_his_6': 0.17798700000000001,
 'item_is_his_pre5_6_sub': 0.17798600000000001,
 'item_is_trade_his': 0.1779356666666667,
 'item_is_trade_his_6': 0.17790933333333334,
 'item_is_trade_his_pre5_6_sub': 0.177982,
 'item_minute_query': 0.1779373333333333,
 'item_minute_query_6': 0.17793733333333331,
 'item_minute_query_pre5_6_sub': 0.17798933333333333,
 'item_occupation_count': 0.17797366666666667,
 'item_occupation_count_6': 0.17796200000000004,
 'item_occupation_count_pre5_6_sub': 0.17795799999999998,
 'item_occupation_trade_count': 0.17777166666666666,
 'item_occupation_trade_count_6': 0.17792233333333332,
 'item_occupation_trade_count_pre5_6_sub': 0.17779233333333333,
 'item_pred_category_score': 0.17794633333333332,
 'item_pred_category_score_item%': 0.17792366666666667,
 'item_pred_category_score_pred%': 0.17799333333333334,
 'item_pred_property_score': 0.17797666666666667,
 'item_pred_property_score_item%': 0.17795799999999998,
 'item_pred_property_score_pred%': 0.17797266666666667,
 'item_property_split_count': 0.17789166666666667,
 'item_score': 0.17798666666666665,
 'item_score2': 0.17797533333333335,
 'item_score2_6': 0.17797533333333335,
 'item_score2_pre5_6_sub': 0.17798933333333333,
 'item_score3': 0.17798899999999998,
 'item_score3_6': 0.17798899999999998,
 'item_score3_pre5_6_sub': 0.17798933333333333,
 'item_score_6': 0.17798666666666665,
 'item_score_pre5_6_sub': 0.17798933333333333,
 'item_trade_hot': 0.17757766666666666,
 'item_trade_hot_6': 0.17780733333333332,
 'item_trade_hot_pre5_6_sub': 0.17763633333333331,
 'item_trade_percent': 0.17728966666666668,
 'item_trade_percent_6': 0.17774299999999998,
 'item_trade_percent_pre5_6_sub': 0.17763499999999999,
 'last_time_sub': 0.17780833333333335,
 'last_time_sub_6': 0.17780833333333335,
 'last_time_sub_full_time': 0.17780833333333335,
 'last_time_sub_pre5_6_sub': 0.17798933333333333,
 'max_click': 0.17671466666666666,
 'max_click_6': 0.17671466666666666,
 'max_click_full_time': 0.17671466666666666,
 'max_click_pre5_6_sub': 0.17798933333333333,
 'max_click_time_sub': 0.17546300000000001,
 'max_click_time_sub_6': 0.17546300000000001,
 'max_click_time_sub_pre5_6_sub': 0.17798933333333333,
 'max_user_item_click': 0.17790133333333336,
 'max_user_item_click_6': 0.17790133333333336,
 'max_user_item_click_full_time': 0.17790133333333336,
 'max_user_item_click_pre5_6_sub': 0.17798933333333333,
 'max_user_item_click_time_sub': 0.176513,
 'max_user_item_click_time_sub_6': 0.176513,
 'max_user_item_click_time_sub_full_time': 0.176513,
 'max_user_item_click_time_sub_pre5_6_sub': 0.17798933333333333,
 'min_time': 0.17795300000000003,
 'min_time_6': 0.17795300000000003,
 'min_time_pre5_6_sub': 0.17798933333333333,
 'minute': 0.17799766666666664,
 'next_time_sub': 0.1746373333333333,
 'next_time_sub_6': 0.174679,
 'next_time_sub_full_time': 0.17472133333333337,
 'next_time_sub_pre5_6_sub': 0.17744800000000002,
 'occupation_hot': 0.1779963333333333,
 'occupation_hot_6': 0.17799633333333331,
 'occupation_hot_pre5_6_sub': 0.17799633333333331,
 'occupation_trade_hot': 0.1779963333333333,
 'occupation_trade_hot_6': 0.17799633333333331,
 'occupation_trade_hot_pre5_6_sub': 0.17799633333333331,
 'occupation_trade_percent': 0.17800233333333335,
 'occupation_trade_percent_6': 0.17799399999999999,
 'occupation_trade_percent_pre5_6_sub': 0.17798900000000004,
 'pred_category_split_count': 0.17798599999999998,
 'pred_property_split_count': 0.1779953333333333,
 'shop_hot': 0.17785666666666666,
 'shop_hot_6': 0.17787733333333333,
 'shop_hot_pre5_6_sub': 0.17788533333333334,
 'shop_item_count': 0.177901,
 'shop_item_count_6': 0.177901,
 'shop_item_count_his': 0.17788,
 'shop_item_count_his_6': 0.17789733333333335,
 'shop_item_count_his_pre5_6_sub': 0.17794633333333332,
 'shop_item_count_pre5_6_sub': 0.17798933333333333,
 'shop_score': 0.17794866666666664,
 'shop_score_6': 0.17794866666666664,
 'shop_score_pre5_6_sub': 0.17798933333333333,
 'shop_trade_hot': 0.1778223333333333,
 'shop_trade_hot_6': 0.17788466666666669,
 'shop_trade_hot_pre5_6_sub': 0.17782000000000001,
 'user_brand_count': 0.17797333333333332,
 'user_brand_count_6': 0.17791633333333334,
 'user_brand_count_pre5_6_sub': 0.17791933333333332,
 'user_brand_trade_count': 0.1779913333333333,
 'user_brand_trade_count_6': 0.17798933333333333,
 'user_brand_trade_count_pre5_6_sub': 0.17797600000000002,
 'user_city_count': 0.17797266666666667,
 'user_city_count_6': 0.17797666666666667,
 'user_city_count_pre5_6_sub': 0.17798366666666665,
 'user_city_trade_count': 0.17798866666666668,
 'user_city_trade_count_6': 0.17798966666666668,
 'user_city_trade_count_pre5_6_sub': 0.17798966666666668,
 'user_context_count': 0.17798933333333333,
 'user_context_count_6': 0.17798933333333333,
 'user_context_count_pre5_6_sub': 0.17798933333333333,
 'user_context_trade_count': 0.17798933333333333,
 'user_context_trade_count_6': 0.17798933333333333,
 'user_context_trade_count_pre5_6_sub': 0.17798933333333333,
 'user_count': 0.17794933333333332,
 'user_count_6': 0.17794933333333332,
 'user_count_first_time_sub': 0.17791433333333337,
 'user_count_first_time_sub_6': 0.17791433333333337,
 'user_count_first_time_sub_full_time': 0.17791433333333337,
 'user_count_first_time_sub_pre5_6_sub': 0.17798933333333333,
 'user_count_full_time': 0.17794933333333332,
 'user_count_pre5_6_sub': 0.17798933333333333,
 'user_day_count': 0.17794933333333332,
 'user_day_count_6': 0.17794933333333332,
 'user_day_count_full_time': 0.17794933333333332,
 'user_day_count_pre5_6_sub': 0.17798933333333333,
 'user_day_query': 0.17671466666666666,
 'user_day_query_6': 0.17671466666666666,
 'user_day_query_pre5_6_sub': 0.17798933333333333,
 'user_hot': 0.17768399999999998,
 'user_hot_6': 0.17795233333333335,
 'user_hot_pre5_6_sub': 0.17775699999999997,
 'user_hour_count': 0.17796900000000002,
 'user_hour_count_6': 0.17796900000000002,
 'user_hour_count_full_time': 0.17796900000000002,
 'user_hour_count_pre5_6_sub': 0.17798933333333333,
 'user_hour_query': 0.17688866666666667,
 'user_hour_query_6': 0.17688866666666667,
 'user_hour_query_pre5_6_sub': 0.17798933333333333,
 'user_is_his': 0.17780666666666667,
 'user_is_his_6': 0.17796933333333331,
 'user_is_his_pre5_6_sub': 0.17798333333333335,
 'user_is_trade_his': 0.17798166666666668,
 'user_is_trade_his_6': 0.17798833333333333,
 'user_is_trade_his_pre5_6_sub': 0.17798100000000003,
 'user_item_count': 0.17794200000000002,
 'user_item_count_6': 0.17790866666666669,
 'user_item_count_first_time_sub': 0.17742999999999998,
 'user_item_count_first_time_sub_6': 0.17742999999999998,
 'user_item_count_first_time_sub_full_time': 0.17742999999999998,
 'user_item_count_first_time_sub_pre5_6_sub': 0.17798933333333333,
 'user_item_count_pre5_6_sub': 0.17782600000000001,
 'user_item_day_num': 0.17745999999999998,
 'user_item_day_num_6': 0.17745999999999998,
 'user_item_day_num_full_time': 0.17745999999999998,
 'user_item_day_num_pre5_6_sub': 0.17798933333333333,
 'user_item_day_query': 0.17790133333333336,
 'user_item_day_query_6': 0.17790133333333336,
 'user_item_day_query_pre5_6_sub': 0.17798933333333333,
 'user_item_hour_query': 0.17796399999999998,
 'user_item_hour_query_6': 0.17796399999999998,
 'user_item_hour_query_pre5_6_sub': 0.17798933333333333,
 'user_item_last_time_sub': 0.17742766666666668,
 'user_item_last_time_sub_6': 0.17742766666666668,
 'user_item_last_time_sub_full_time': 0.17742766666666668,
 'user_item_last_time_sub_pre5_6_sub': 0.17798933333333333,
 'user_item_minute_query': 0.17798933333333333,
 'user_item_minute_query_6': 0.17798933333333333,
 'user_item_minute_query_pre5_6_sub': 0.17798933333333333,
 'user_item_next_time_sub': 0.17676,
 'user_item_next_time_sub_6': 0.17666633333333334,
 'user_item_next_time_sub_full_time': 0.176728,
 'user_item_next_time_sub_pre5_6_sub': 0.17775233333333332,
 'user_item_num': 0.17745999999999998,
 'user_item_num_6': 0.17745999999999998,
 'user_item_num_full_time': 0.17745999999999998,
 'user_item_num_pre5_6_sub': 0.17798933333333333,
 'user_item_trade_count': 0.177984,
 'user_item_trade_count_6': 0.17798933333333333,
 'user_item_trade_count_pre5_6_sub': 0.17799666666666666,
 'user_minute_count': 0.17791966666666667,
 'user_minute_count_6': 0.17791966666666667,
 'user_minute_count_full_time': 0.17791966666666667,
 'user_minute_count_pre5_6_sub': 0.17798933333333333,
 'user_minute_query': 0.176573,
 'user_minute_query_6': 0.17657300000000001,
 'user_minute_query_pre5_6_sub': 0.17798933333333333,
 'user_query_day': 0.17671466666666666,
 'user_query_day_hour': 0.17688866666666667,
 'user_query_minute': 0.176573,
 'user_sell_power': 0.17770100000000003,
 'user_sell_power_6': 0.1779466666666667,
 'user_sell_power_mean': 0.17777566666666667,
 'user_sell_power_mean_6': 0.17796866666666666,
 'user_sell_power_mean_item_minus': 0.17779,
 'user_sell_power_mean_item_minus_6': 0.17796833333333331,
 'user_sell_power_mean_item_minus_pre5_6_sub': 0.17784266666666668,
 'user_sell_power_mean_pre5_6_sub': 0.17785833333333334,
 'user_sell_power_pre5_6_sub': 0.17777666666666667,
 'user_sell_trade_power': 0.178001,
 'user_sell_trade_power_6': 0.17800166666666664,
 'user_sell_trade_power_mean': 0.1779926666666667,
 'user_sell_trade_power_mean_6': 0.17800099999999999,
 'user_sell_trade_power_mean_item_minus': 0.17798966666666668,
 'user_sell_trade_power_mean_item_minus_6': 0.17799466666666666,
 'user_sell_trade_power_mean_item_minus_pre5_6_sub': 0.17799266666666669,
 'user_sell_trade_power_mean_pre5_6_sub': 0.17799466666666666,
 'user_sell_trade_power_pre5_6_sub': 0.17799499999999999,
 'user_shop_count': 0.17795366666666668,
 'user_shop_count_6': 0.17789533333333332,
 'user_shop_count_first_time_sub': 0.17751433333333333,
 'user_shop_count_first_time_sub_6': 0.17751433333333333,
 'user_shop_count_first_time_sub_full_time': 0.17751433333333333,
 'user_shop_count_first_time_sub_pre5_6_sub': 0.17798933333333333,
 'user_shop_count_pre5_6_sub': 0.1778763333333333,
 'user_shop_day_num': 0.17754066666666665,
 'user_shop_day_num_6': 0.17754066666666665,
 'user_shop_day_num_full_time': 0.17754066666666665,
 'user_shop_day_num_pre5_6_sub': 0.17798933333333333,
 'user_shop_last_time_sub': 0.177509,
 'user_shop_last_time_sub_6': 0.177509,
 'user_shop_last_time_sub_full_time': 0.177509,
 'user_shop_last_time_sub_pre5_6_sub': 0.17798933333333333,
 'user_shop_next_time_sub': 0.17701500000000001,
 'user_shop_next_time_sub_6': 0.17692566666666665,
 'user_shop_next_time_sub_full_time': 0.17696733333333334,
 'user_shop_next_time_sub_pre5_6_sub': 0.17777600000000002,
 'user_shop_num': 0.17754066666666665,
 'user_shop_num_6': 0.17754066666666665,
 'user_shop_num_full_time': 0.17754066666666665,
 'user_shop_num_pre5_6_sub': 0.17798933333333333,
 'user_shop_trade_count': 0.17798633333333333,
 'user_shop_trade_count_6': 0.17798933333333333,
 'user_shop_trade_count_pre5_6_sub': 0.17799366666666669,
 'user_trade_hot': 0.17798433333333333,
 'user_trade_hot_6': 0.17799900000000002,
 'user_trade_hot_pre5_6_sub': 0.17797566666666667,
 'user_trade_percent': 0.17798033333333332,
 'user_trade_percent_6': 0.17799066666666666,
 'user_trade_percent_pre5_6_sub': 0.17798766666666666,
 'user_yesterday_query': 0.17798933333333333,
 'user_yesterday_query_6': 0.17798933333333333,
 'user_yesterday_query_pre5_6_sub': 0.17798933333333333,
 'user_yesterhour_query': 0.17800066666666667,
 'user_yesterhour_query_6': 0.17800066666666667,
 'user_yesterhour_query_pre5_6_sub': 0.17798933333333333,
 'yesterday': 0.17798933333333333,
 'yesterday_6': 0.17798933333333333,
 'yesterday_pre5_6_sub': 0.17798933333333333,
 'yesterhour': 0.17797366666666667,
 'yesterhour_6': 0.17797366666666667,
 'yesterhour_pre5_6_sub': 0.17798933333333333}